In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable
import numpy as np 
import matplotlib.pyplot as plt
import functions as f
import cv2
from classes import Discriminator, Generator, weights_init
from PIL import Image

In [ ]:
device = torch.device("cuda: 0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
#Inititate constants
PROCESS_DATA = 1 #Change to 1 when training
TRAINING_DATA = 1 #Change to 1 if want model to be trained again

In [ ]:
#Inititate constants
BATCH_SIZE = 64
IMG_SIZE = 64

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [ ]:
if PROCESS_DATA:
    dataset = datasets.FashionMNIST("data",train = True,transform=data_transform,download = True)
    trainloader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers = 4, shuffle=True)

In [ ]:
#Show training data size
print(len(dataset))

In [ ]:
#Show training labels
LABELS = dataset.classes
print(LABELS)

In [ ]:
#Visualize some data samples
def visualize_samples(NUM_PIC=16):
    a = np.random.randint(BATCH_SIZE-NUM_PIC)
    fig = plt.figure(figsize=(10,10))
    img = next(iter(trainloader))
    plt.axis("off")
    plt.title("Training Images")
    plt.imshow(np.transpose(torchvision.utils.make_grid(img[0][a:a+NUM_PIC],padding=2, normalize=True),(1,2,0)))
    plt.show()

visualize_samples(16)

In [ ]:
LATENT_SIZE = 100

In [ ]:
#Make latent vector by creating random Gaussian noise
def random_noise(size):
    noise = torch.randn(size,LATENT_SIZE,1,1)
    noise = Variable(noise).to(device)
    return noise

In [ ]:
#Generate labels for loss fuction
def generate_labels(size, label):
    if label:
        data = torch.ones(size,1)-0.1
    else:
        data = torch.zeros(size,1)
    data = Variable(data).view(-1).to(device)
    return data

In [ ]:
#Inititate weights for Generator and Discriminator with mean = 0.02, standard deviation = 1
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
#Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100,128*8,4,1,0),
            nn.BatchNorm2d(128*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(128*8,128*4,4,2,1),
            nn.BatchNorm2d(128*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(128*4,128*2,4,2,1),
            nn.BatchNorm2d(128*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(128*2,128,4,2,1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128,1,4,2,1),
            nn.Tanh()
        )
    def forward(self, x):
        x = self.main(x)
        return x

In [ ]:
#Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1,128,4,2,1, bias=False),
            nn.LeakyReLU(0.2, inplace= True),

            nn.Conv2d(128,128*2,4,2,1, bias=False),
            nn.BatchNorm2d(128*2),
            nn.LeakyReLU(0.2, inplace= True),

            nn.Conv2d(128*2,128*4,4,2,1, bias=False),
            nn.BatchNorm2d(128*4),
            nn.LeakyReLU(0.2, inplace= True),

            nn.Conv2d(128*4,128*8,4,2,1, bias=False),
            nn.BatchNorm2d(128*8),
            nn.LeakyReLU(0.2, inplace= True),

            nn.Conv2d(128*8,1,4,1,0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
def train_discriminator(D, optimizer, error, real_data, fake_data):
    batch_size = real_data.size(0)
    optimizer.zero_grad()

    real_data = real_data.to(device)
    fake_data = fake_data.to(device)

    prediction_real = D(real_data).view(-1)
    real_label = generate_labels(batch_size,1)
    loss_real = error(prediction_real,real_label)
    loss_real.backward()

    prediction_fake = D(fake_data).view(-1)
    fake_label = generate_labels(batch_size,0)
    loss_fake = error(prediction_fake, fake_label)
    loss_fake.backward()

    D_x = prediction_real.mean().item()
    D_g_z1 = prediction_fake.mean().item()
    optimizer.step()

    return loss_real.data + loss_fake.data, D_x, D_g_z1

In [ ]:
def train_generator(D, optimizer, error, data):
    batch_size = data.size(0)
    optimizer.zero_grad()
    data = data.to(device)
    prediction = D(data).view(-1)
    label = generate_labels(batch_size, 1)
    loss = error(prediction, label)
    loss.backward()
    D_g_z2 = prediction.mean().item()
    optimizer.step()
    return loss.data, D_g_z2

In [ ]:
def generate_img(G,noise):
    G.eval()
    with torch.no_grad():
        fig = plt.figure(figsize=(10,10))
        test_img = G(noise).view(-1,1,IMG_SIZE,IMG_SIZE).data.cpu()
        img = np.transpose(torchvision.utils.make_grid(test_img[:],padding=2, normalize=True),(1,2,0))
        img = np.array(img)
        plt.imshow(img)
        plt.axis("off")
        plt.title("Generated Images")
        plt.show()
    G.train()

In [ ]:
#Generate latent vector
test_noise = random_noise(16)

In [ ]:
def train(D, G, D_optimizer, G_optimizer, error, trainloader):
    EPOCHS = 5
    D_loss_list = []
    G_loss_list = []
    print("Start Training.....")
    for epoch in range(EPOCHS):
        for i, (real_batch,_) in enumerate(trainloader):  
            batch_size = real_batch.size(0)
            fake_data = G(random_noise(batch_size)).detach()
            real_data = Variable(real_batch.view(-1,1,IMG_SIZE,IMG_SIZE))
            D_loss,D_x, D_g_z1 = train_discriminator(D,D_optimizer,error,real_data,fake_data)
            data = G(random_noise(batch_size))
            G_loss, D_g_z2 = train_generator(D,G_optimizer,error,data)
            if i%50 == 0 :
                print("Epoch: ({}/{}), Batch: ({}/{}), D_Loss: {:.4f}, G_Loss: {:.4f}, D(x): {:.4f}, G(D(z)): {:.4f} / {:.4f}".format(epoch,EPOCHS,i,len(trainloader),D_loss,G_loss,D_x, D_g_z1,D_g_z2))
            D_loss_list.append(D_loss)
            G_loss_list.append(G_loss)   
        torch.save(G.state_dict(), "model/generator-dcgan2.pth")
        torch.save(D.state_dict(), "model/discriminator-dcgan2.pth")
        generate_img(G,test_noise)
    #plot(D_loss_list,G_loss_list,len(D_loss_list))
    return G, D_loss_list,G_loss_list

In [ ]:
D = Discriminator().to(device)
G = Generator().to(device) 
D.apply(weights_init)
G.apply(weights_init)

if TRAINING_DATA:
    D_optimizer = torch.optim.Adam(D.parameters(), lr= 0.0002, betas=(0.5, 0.999))
    G_optimizer = torch.optim.Adam(G.parameters(), lr = 0.0002, betas=(0.5, 0.999))
    error = nn.BCELoss()
    G, D_loss_list, G_loss_list = train(D,G,D_optimizer,G_optimizer,error,trainloader)
else:
    G.load_state_dict(torch.load("model/generator-dcgan2.pth"))


In [ ]:
def plot(D_loss_list, G_loss_list, EPOCHS=None):
    if EPOCHS is None:
        EPOCHS = len(D_loss_list)
    plt.subplot(2,1,1)
    plt.plot(range(EPOCHS), D_loss_list)
    plt.title("Discriminator Loss")
    plt.subplot(212)
    plt.plot(range(EPOCHS),G_loss_list)
    plt.title("Gennerator Loss")
    plt.show()
    
plot(D_loss_list, G_loss_list)

In [ ]:
test_noise = f.random_noise(16)
generate_img(G,test_noise)